In [28]:
import pandas as pd
from sklearn import datasets
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html

In [29]:
iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df["species"] = iris.target_names[iris.target]
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [54]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
server = app.server
indicator = list(df.columns)
app.layout = html.Div([
    html.Div([
        html.Div([
            html.Label("X Axis"),
            dcc.Dropdown(id="x-axis-filter",
                        options=[{"label":i, "value":i} for i in indicator],
                         value=indicator[0]),
            dcc.RadioItems(id="x-axis-scale",
                          options=[{"label":i, "value":i} for i in ["Linear", "Log"]],
                          value="Linear")
        ],
        style={"width":"49%", "display":"inline-block"}),
        html.Div([
            html.Label("Y Axis"),
            dcc.Dropdown(id="y-axis-filter",
                        options=[{"label":i, "value":i} for i in indicator],
                         value=indicator[1]),
            dcc.RadioItems(id="y-axis-scale",
                          options=[{"label":i, "value":i} for i in ["Linear", "Log"]],
                          value="Linear")            
        ],
        style={"width":"49%", "float":"right", "display":"inline-block"}),            
    ]),
    html.Div([
        dcc.Graph(id="scatter-plot")
    ],
    style={"width":"98%", "display":"inline-block"}),
    html.Div([
        html.H4("Bar Plot")
    ]),
    html.Div([
        dcc.Dropdown(id="bar-filter",
                        options=[{"label":i, "value":i} for i in indicator],
                         value=indicator[1])
    ]),
    html.Div([
        dcc.Graph(id="bar-plot")
    ])
])

@app.callback(
    dash.dependencies.Output("scatter-plot", "figure"),
    [dash.dependencies.Input("x-axis-filter", "value"),
    dash.dependencies.Input("y-axis-filter", "value"),
    dash.dependencies.Input("x-axis-scale", "value"),
    dash.dependencies.Input("y-axis-scale", "value"),
    ]
)
def update_graph(x_axis_value, y_axis_value, x_axis_scale, y_axis_scale):
    log_dic = {"Linear":False, "Log":True}
    fig = px.scatter(df, x=x_axis_value, y=y_axis_value, color="species", marginal_y="violin",
           marginal_x="box", log_x=log_dic[x_axis_scale],log_y=log_dic[y_axis_scale],trendline="ols", template="simple_white")    
    return fig
    
@app.callback(
    dash.dependencies.Output("bar-plot", "figure"),
    dash.dependencies.Input("bar-filter", "value")
)
def update_bar(bar_filter):
    fig = px.histogram(df, x=bar_filter)    
    return fig

In [55]:
app.run_server(mode="jupyterlab", port=8051)